In [113]:
# coding:utf8
from PIL import Image, ImageDraw, ImageFont
from PIL.ImageOps import invert
import matplotlib.pyplot as plt
import numpy as np
import math
import csv
import os
import codecs

In [114]:
def create_folder(dir, folder_name):

    output_dir = os.path.join(dir, folder_name)

    if not os.path.isdir(output_dir):
        os.mkdir(output_dir)
    return output_dir

In [115]:
def monochrome(input_array):
    height, width = input_array.shape[:2]
    output_array = np.zeros((height, width), dtype=input_array.dtype)

    for y in range(height):
        for x in range(width):
            output_array[y][x] = np.mean(input_array[y][x])

    return output_array
  
def binarization_b1w0(input_array):
    height, width = input_array.shape[:2]
    input_array = monochrome(input_array)
    # print(np.shape(input_array)[:])
    output_array = np.zeros((height, width), int)

    for i in range(height):
        for j in range(width):
            if input_array[i][j] >= 200:
                output_array[i][j] = 1
            else:
                output_array[i][j] = 0
    
    return output_array

def first_nonzero(arr):
    first_non0 = -1
    for i in range (np.shape(arr)[0]):
        if arr[i] > 0:
            first_non0 = i
            break

    return first_non0

def last_nonzero(arr):
    arr_reversed = np.flip(arr) 
    first_non0 = -1

    for i in range (np.shape(arr)[0]):
        if arr_reversed[i] > 0:
            first_non0 = np.shape(arr)[0] - i #!!!!!!!!!!!!!!!!!!
            break
    return first_non0


In [116]:
def profile_y(input_array):
    return {'y_axis' : np.arange(0, input_array.shape[0]).astype(int), # разметка оси y, 0 тк первое значение размера это колво строк
            'values' : np.sum(input_array, axis=1) # проекция на ось y
            }

def profile_x(input_array):
    return {'values' : np.sum(input_array, axis=0), # проекция на ось x
            'x_axis' : np.arange(0, input_array.shape[1]).astype(int) # разметка оси x
            }


def save_profiles_as_graphics(img, letter, path):
    x = profile_x(img)
    y = profile_y(img)

    for axis in x.items():
        plt.bar(x['x_axis'], x['values'], color='blue')
        plt.xlabel('Сумма значений')  # Подпись оси X
        plt.ylabel('Пиксели')  # Подпись оси Y
        plt.savefig(os.path.join( path, f'profile_{letter}_axis_x.png'))  # Сохранение диаграммы как изображения bmp
        plt.close()  # Очистка

    for axis in y.items():
        plt.barh(y['y_axis'], y['values'], color='blue') # Используйте профили для создания столбчатой диаграммы
        plt.xlabel('Сумма значений')  # Подпись оси X
        plt.ylabel('Пиксели')  # Подпись оси Y
        plt.savefig(os.path.join( path, f'profile_{letter}_axis_y.png'))  # Сохранение диаграммы как изображения bmp
        plt.close()  # Очистка

def X_diagram(input_array):
    
    height, width = input_array.shape[:2]
    p = profile_x(input_array)
    output_array = np.zeros((height, width), dtype=input_array.dtype)

    
    for x in range(width):
        for y in range(height):
            if y+1 <= p['values'][x]:
                output_array[y][x] = 0
            else:
                output_array[y][x] = 255

    # new_image = Image.fromarray(output_array.astype(np.uint8))
    # new_image.save('NEW0.bmp')
    return output_array

    
def Y_diagram(input_array):
    height, width = input_array.shape[:2]

    p = profile_y(input_array)
    output_array = np.zeros((height, width), dtype=input_array.dtype)

    for x in range(width):
        for y in range(height):
            if x+1 <= p['values'][y]:
                output_array[y][x] = 0
            else:
                output_array[y][x] = 255
    return output_array

# def edit_img(img):
#     img_array = np.array(img)
#     # print("a", np.shape(img_array))
#     img_array_01 = binarization_b1w0(img_array)
#     # img_array_cut = img_array_01[first_nonzero( profile_y(img_array_01)['values'] ) : last_nonzero( profile_y(img_array_01)['values'] ),
#     #                              first_nonzero( profile_x(img_array_01)['values'] ) : last_nonzero( profile_x(img_array_01)['values'] )]
#     # # print("b", np.shape(img_array_cut))
#     return img_array_cut

def edit_img2(letter,path, DIR_negative_letters_imgs):
    input_image = Image.open(os.path.join(DIR_negative_letters_imgs, f'{letter}.bmp'))
    img_array = np.array(input_image)

    outpt = create_folder(path, "edited")
    neg = create_folder(outpt, "neg")
    # print(neg)
    pos = create_folder(outpt, "pos")

    img_array_01 = binarization_b1w0(img_array)
    img_array_cut = img_array[first_nonzero( profile_y(img_array_01)['values'] ) : last_nonzero( profile_y(img_array_01)['values'] ),
                                 first_nonzero( profile_x(img_array_01)['values'] ) : last_nonzero( profile_x(img_array_01)['values'] )]

    new_image = Image.fromarray(img_array_cut.astype(np.uint8))
    new_image.save(os.path.join(neg, f'{letter}.bmp'))

    new_image1 = np.invert(img_array_cut)
    new_image2 = Image.fromarray(new_image1.astype(np.uint8))
    new_image2.save(os.path.join(pos, f'{letter}.bmp'))


def XY_diagram_join(letter, path, DIR_normal_letters_imgs, DIR_negative_letters_imgs):

    PATH_image = os.path.join(DIR_negative_letters_imgs, f'{letter}.bmp')
    image = Image.open(PATH_image)
    input_image = np.array(image)

    PATH_image = os.path.join(DIR_normal_letters_imgs, f'{letter}.bmp')
    image = Image.open(PATH_image)
    img_array1 = np.array(image)
    
    input_image_mono = monochrome(img_array1)
    input_image_bin  = binarization_b1w0(input_image)

    Y_array = Y_diagram(input_image_bin)
    X_array = X_diagram(input_image_bin)

    height, width = input_image_bin.shape[:2]

    zero_array  = np.ones((height,width),  dtype=input_image_bin.dtype)*255

    # print(np.shape(input_image),
    #       np.shape(X_array),
    #       np.shape(Y_array),
    #       np.shape(zero_array)
    #       )
    
    final0 = np.vstack([Y_array, zero_array])
    final1 = np.vstack([input_image_mono, X_array])
    final = np.hstack([final1, final0])   

    new_image = Image.fromarray(final.astype(np.uint8))
    new_image.save(os.path.join(path, f'{letter}.bmp'))

In [117]:
def create_image_from_letter(font_path, letter, dir_normal_letters_imgs, dir_negative_letters_imgs):
  image = Image.new('RGB', (600, 600), color=(255, 255, 255))
  font = ImageFont.truetype(font_path, size=500)
  
  # Создаем объект для рисования текста
  draw = ImageDraw.Draw(image)

  # Рисуем букву в центре изображения
  draw.text((300, 300), letter, font=font, fill=(0, 0, 0), anchor="mm")
  image.save(os.path.join(dir_normal_letters_imgs, f'{letter}.bmp'))
  img = invert(image)
  img.save(os.path.join(dir_negative_letters_imgs, f'{letter}.bmp'))


In [118]:
def get_characteristics(img_edited):

    # Вес (масса чёрного) каждой четверти изображения символа
    weight = img_edited.sum()

    # Удельный вес (вес, нормированный к четверти площади)
    rel_weight = weight / (img_edited.shape[0] * img_edited.shape[1])

    # Координаты центра тяжести
    x_avg = 0
    for x, column in enumerate(img_edited.T):
        x_avg += sum((x + 1) * column)
    x_avg = x_avg / weight

    y_avg = 0
    for y, row in enumerate(img_edited):
        y_avg += sum((y + 1) * row)
    y_avg = y_avg / weight
    
    # Нормированные координаты центра тяжести
    rel_x_avg = (x_avg - 1) / (img_edited.shape[1] - 1)
    rel_y_avg = (y_avg - 1) / (img_edited.shape[0] - 1)

    # Осевые моменты инерции по горизонтали и вертикали
    iner_x = 0
    for y, row in enumerate(img_edited):
        iner_x += sum((y + 1 - y_avg) * 2 * row)

    iner_y = 0
    for x, column in enumerate(img_edited.T):
        iner_y += sum((x + 1 - x_avg) * 2 * column)

    # Нормированные осевые моменты инерции
    rel_iner_x = iner_x / (img_edited.shape[0] * 2 + img_edited.shape[1] * 2)
    rel_iner_y = iner_y / (img_edited.shape[0] * 2 + img_edited.shape[1] * 2)

    return {
        'weight': weight,
        'rel_weight': rel_weight,
        'center': [x_avg, y_avg],
        'rel_center': [rel_x_avg, rel_y_avg],
        'inertia': [iner_x, iner_y],
        'rel_inertia': [rel_iner_x, rel_iner_y]
    }

In [119]:
def save_characteristics_to_csv(letter, characteristics_list, output_file):
    

    with open(output_file, 'a', newline='') as csvfile:
        fields = [ 'Вес_чёрного', 'Норм_вес_чёрного', 'Центр_массы', 'Нормированный_центр_массы', 'Моменты_инерции', 'Нормированные_моменты_инерции']
        
        csvwriter = csv.DictWriter(csvfile, fieldnames=fields)
        csvwriter.writeheader()
        # csvwriter.writerow(characteristics_list)

        center_mass_x, center_mass_y = characteristics_list['center']
        normalized_center_mass_x, normalized_center_mass_y = characteristics_list['rel_center']
        normalized_center_mass_x, normalized_center_mass_y = characteristics_list['rel_center']
        normalized_inertia_x, normalized_inertia_y = characteristics_list['rel_inertia']
        inertia_x, inertia_y = characteristics_list['inertia']

        csvwriter.writerow({
            'Буква': letter,
            'Вес_чёрного': f"{characteristics_list['weight']}",
            'Норм_вес_чёрного': f"{characteristics_list['rel_weight']}",
            'Центр_массы': f"({center_mass_x}, {center_mass_y})",
            'Нормированный_центр_массы': f"({normalized_center_mass_x}, {normalized_center_mass_y})",
            'Моменты_инерции': f"({inertia_x}, {inertia_y})",
            'Нормированные_моменты_инерции': f"({normalized_inertia_x}, {normalized_inertia_y})"
            })

        

In [120]:

def main():
    letters = ['A','B','C','Ç','D','E','F',
               'G','Ğ','H','I','İ','J','K',
               'L','M','N','O','Ö','P','R',
               'S','Ş','T','U','Ü','V','Y','Z']
 
    PATH_font = "c:/Users/Milana/Documents/lab5/input/calibri.ttf"
    DIR_cwd = os.getcwd()
    DIR_output = create_folder(DIR_cwd, "output")
    # DIR_output2 = create_folder(DIR_output, "edited")
    DIR_output_im = create_folder(DIR_output, "images")
    DIR_hitst = create_folder(DIR_output, "hists")
    DIR_normal_letters_imgs = create_folder(DIR_output, "normal")
    DIR_negative_letters_imgs = create_folder(DIR_output, "negative")

    for letter in letters:
        # print(letter)
        create_image_from_letter(PATH_font, letter, DIR_normal_letters_imgs, DIR_negative_letters_imgs)
        edit_img2(letter, DIR_output, DIR_negative_letters_imgs)

        PATH_image = os.path.join('C:/Users/Milana/Documents/oavi/lab5/output/edited/neg', f'{letter}.bmp')
        image = Image.open(PATH_image)
        img_array1 = np.array(image)
        img_array_b0w1 = binarization_b1w0(img_array1)

        characteristics_list = get_characteristics(img_array_b0w1)
        save_characteristics_to_csv(letter, characteristics_list, os.path.join(DIR_output, "output.csv"))
        save_profiles_as_graphics(img_array_b0w1, letter, DIR_hitst)
        # print(letter)

        XY_diagram_join(letter, DIR_output_im, 'C:/Users/Milana/Documents/oavi/lab5/output/edited/pos','C:/Users/Milana/Documents/oavi/lab5/output/edited/neg')


if __name__ == '__main__':
    main()